In [0]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split


In [2]:
monday = pd.read_csv('Monday-WorkingHours.pcap_ISCX.csv')
tuesday = pd.read_csv('Tuesday-WorkingHours.pcap_ISCX.csv')
wednesday = pd.read_csv('Wednesday-workingHours.pcap_ISCX.csv')
thursday_morning = pd.read_csv('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
thursday_afternoon = pd.read_csv('Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
friday_morning = pd.read_csv('Friday-WorkingHours-Morning.pcap_ISCX.csv')
friday = pd.read_csv('Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
monday = monday.dropna()
tuesday = tuesday.dropna()
wednesday = wednesday.dropna()
thursday_morning = thursday_morning.dropna()
thursday_afternoon = thursday_afternoon.dropna()
friday_morning = friday_morning.dropna()
friday = friday.dropna()

In [0]:
# clean data
convert_dict = {' Subflow Bwd Packets': int} 
monday = monday.astype(convert_dict)

In [0]:
days = pd.concat([monday, tuesday, wednesday, thursday_morning, thursday_afternoon, friday_morning, friday])


In [0]:
column_names = list(days.columns.values)
processed_names = []
for name in column_names:
    processed_names.append(name.strip())

In [0]:
days.columns = processed_names

convert_dict = {'Flow Packets/s': np.float64,
                'Flow Bytes/s': np.float64} 

In [0]:
days = days.astype(convert_dict)


In [0]:
days = days.replace([np.inf], np.nan).dropna()


In [0]:
days = days.drop(['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 
                  'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'], axis=1)

In [0]:
days = shuffle(days)


In [0]:
labels = days['Label']
days = days.drop('Label', axis=1)

In [13]:
days.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06,2.602165e+06
mean,7.990839e+03,1.467545e+07,9.758744e+00,1.090970e+01,5.160453e+02,1.706530e+04,1.791100e+02,1.793506e+01,4.901032e+01,5.631750e+01,7.100250e+02,4.320342e+01,2.555662e+02,2.580603e+02,1.570333e+06,7.576483e+04,1.275386e+06,2.807129e+06,8.818802e+06,1.742039e+05,1.441948e+07,2.619071e+06,3.103233e+06,8.709989e+06,1.093668e+06,1.019341e+07,1.882197e+06,1.476805e+06,4.699958e+06,1.032640e+06,4.754195e-02,1.210530e-04,-2.829077e+04,-2.482236e+03,6.838098e+04,7.467238e+03,1.717538e+01,7.540120e+02,1.423652e+02,2.267059e+02,2.868267e+05,3.811326e-02,4.754195e-02,2.532507e-04,2.936924e-01,2.989126e-01,9.084474e-02,1.210530e-04,2.544035e-04,6.559065e-01,1.589945e+02,4.901032e+01,2.555662e+02,-2.829077e+04,9.758744e+00,5.160339e+02,1.090970e+01,1.706493e+04,7.230451e+03,2.108637e+03,5.606678e+00,-2.984414e+03,7.268074e+04,4.362531e+04,1.485864e+05,4.800757e+04,8.151055e+06,2.347563e+05,8.342386e+06,7.942940e+06
std,1.813875e+04,3.384397e+07,7.818916e+02,1.040251e+03,1.037854e+04,2.360362e+06,4.977089e+02,4.045666e+01,1.203394e+02,1.698114e+02,1.613022e+03,6.985559e+01,5.072902e+02,6.563974e+02,2.657912e+07,2.624696e+05,4.632946e+06,8.075050e+06,2.423088e+07,3.068702e+06,3.375477e+07,9.779912e+06,9.519382e+06,2.423685e+07,8.887249e+06,2.924457e+07,9.166357e+06,6.346399e+06,1.725209e+07,8.581915e+06,2.127950e-01,1.100175e-02,2.195805e+07,1.514649e+06,2.556205e+05,3.932311e+04,2.577736e+01,1.650725e+03,2.515791e+02,4.852088e+02,9.912441e+05,1.914697e-01,2.127950e-01,1.591184e-02,4.554528e-01,4.577816e-01,2.873882e-01,1.100175e-02,1.594801e-02,5.621446e-01,2.682781e+02,1.203394e+02,5.072902e+02,2.195805e+07,7.818916e+02,1.036438e+04,1.040251e+03,2.360330e+06,1.473652e+04,8.710950e+03,6.637795e+02,1.131640e+06,6.335535e+05,4.054992e+05,1.036407e+06,5.546051e+05,2.378127e+07,2.831947e+06,2.411020e+07,2.365606e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00000

In [0]:
labels[labels.str.contains('Web Attack')] = 'Web Attack'
labels[labels.str.contains('DoS')] = 'DoS'
labels[labels.str.contains('Heartbleed')] = 'DoS'
labels[labels.str.contains('BENIGN')] = 'Normal'
labels[labels.str.contains('Patator')] = 'Brute Force'

In [15]:
labels.unique()

array(['PortScan', 'Normal', 'DoS', 'Brute Force', 'Bot', 'Web Attack',
       'Infiltration'], dtype=object)

In [16]:
labels.value_counts()/len(labels)

Normal          0.835318
DoS             0.096736
PortScan        0.061028
Brute Force     0.005316
Web Attack      0.000838
Bot             0.000752
Infiltration    0.000014
Name: Label, dtype: float64

In [0]:
days=(days-days.mean())/(days.std())

In [0]:
ohe_labels = pd.get_dummies(list(labels))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(days, ohe_labels, test_size = 0.1)


In [49]:
X_train_m = X_train.values
y_train_m = y_train.values

X_test_m = X_test.values
y_test_m = y_test.values

num_classes = y_train_m.shape[1]
n_samples = y_train_m.shape[0]
num_features = X_train_m.shape[1]
print(X_train.shape)
print(y_train_m.shape)
print(X_test_m.shape)
print(y_test_m.shape)
num_output = y_train_m.shape[1]
num_layers_0 = 50
regularizer_rate = 0.1


(2341948, 70)
(2341948, 7)
(260217, 70)
(260217, 7)


In [0]:
# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 100
display_step = 1

In [0]:
# Network Parameters
n_hidden_1 = 50
n_hidden_2 = 25 
n_input = num_features 
n_classes = num_classes

In [61]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])


# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]),stddev=(1/tf.sqrt(float(num_features)))),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), stddev=(1/tf.sqrt(float(num_features)))),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), stddev=(1/tf.sqrt(float(num_features))))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


# Construct model
logits = multilayer_perceptron(X)


# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples/batch_size)
        for batch in range(total_batch):
            batch_x = X_train_m[batch*batch_size : (1+batch)*batch_size]
            batch_y = y_train_m[batch*batch_size : (1+batch)*batch_size]
    
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
        
            avg_cost += c / total_batch

        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")
    
    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: X_test_m, Y: y_test_m}))

Epoch: 0001 cost=1.512622742
Epoch: 0002 cost=0.225814953
Epoch: 0003 cost=0.187453728
Epoch: 0004 cost=0.169064487
Epoch: 0005 cost=0.147159513
Epoch: 0006 cost=0.137750393
Epoch: 0007 cost=0.128440935
Epoch: 0008 cost=0.110048325
Epoch: 0009 cost=0.106347157
Epoch: 0010 cost=0.100715927
Epoch: 0011 cost=0.100039973
Epoch: 0012 cost=0.089910171
Epoch: 0013 cost=0.085539534
Epoch: 0014 cost=0.086279831
Epoch: 0015 cost=0.084002864
Epoch: 0016 cost=0.081058796
Epoch: 0017 cost=0.078312054
Epoch: 0018 cost=0.078787080
Epoch: 0019 cost=0.078361778
Epoch: 0020 cost=0.076463821
Epoch: 0021 cost=0.074371401
Epoch: 0022 cost=0.073041891
Epoch: 0023 cost=0.074486022
Epoch: 0024 cost=0.072454924
Epoch: 0025 cost=0.072320380
Epoch: 0026 cost=0.071720970
Epoch: 0027 cost=0.069980787
Epoch: 0028 cost=0.070053586
Epoch: 0029 cost=0.069590361
Epoch: 0030 cost=0.069417451
Epoch: 0031 cost=0.069282449
Epoch: 0032 cost=0.067589440
Epoch: 0033 cost=0.067511018
Epoch: 0034 cost=0.067880853
Epoch: 0035 co